In [ ]:
# importação das bibliotecas
import mysql.connector as connection
import pandas as pd
import plotly.express as px

In [ ]:
# conexão com mysql e criação do pandas dataframe
try:
    mydb = connection.connect(
        host='localhost', 
        database = 'combustivel',
        user='root', 
        passwd='familiaunida420')
    
    query_2020 = 'select * from combustivel_automotivo_2020;'
    df_2020 = pd.read_sql(query_2020, mydb)

    query_2021 = 'select * from combustivel_automotivo_2021;'
    df_2021 = pd.read_sql(query_2021, mydb)

    mydb.close()
except Exception as e:
    mydb.close()
    print(str(e))

df = pd.concat([df_2020, df_2021])

In [ ]:
# criação de colunas no dataframe
df['ano_data_coleta'] = pd.to_datetime(df.data_coleta).dt.to_period('Y').astype(str)
df['mes_data_coleta'] = pd.to_datetime(df.data_coleta).dt.to_period('M').astype(str)

In [ ]:
# visualização dos dados no dataframe
df.head()

In [ ]:
# funções auxiliares para criações dos gráficos 
def groupby_mean(df, groupby, mean):
    df_groupby = df.groupby(groupby).mean(mean).reset_index()
    return df_groupby

def groupby_count(df, groupby):
    df_groupby = df.groupby(groupby).size().reset_index(name='count')
    return df_groupby

In [ ]:
# bar chart - média do valor de venda por uf
media_valor_venda_uf = groupby_mean(df, ['regiao_sigla', 'uf', 'ano_data_coleta'], 'valor_venda')

fig = px.line(media_valor_venda_uf, x='uf', y='valor_venda', color='ano_data_coleta',
            hover_data=['regiao_sigla'], title='Média do Valor de Venda por UF')
fig.update_layout(yaxis_range = [0.1, 7])
fig.show()

In [ ]:
# line chart - média do valor de venda por dia
media_valor_venda_data = groupby_mean(df, ['data_coleta', 'mes_data_coleta'], 'valor_venda')

fig = px.line(media_valor_venda_data, x='data_coleta', y='valor_venda', title='Média do Valor de Venda por Dia')
fig.update_layout(yaxis_range = [0.1, 7])
fig.show()

In [ ]:
# line chart - média do valor de venda por mês
media_valor_venda_mes = groupby_mean(df, 'mes_data_coleta', 'valor_venda')

fig = px.line(media_valor_venda_mes, x='mes_data_coleta', y='valor_venda', title='Média do Valor de Venda por Mês')
fig.update_layout(yaxis_range = [0.1, 7])
fig.show()

In [ ]:
# boxplot - dispersão dos valores de venda por uf
dispersao_valor_venda_uf = df[['regiao_sigla', 'uf', 'produto', 'valor_venda']]

fig = px.box(dispersao_valor_venda_uf, x='uf', y='valor_venda', title='Dispersão dos Valores de Venda por UF')
fig.show()

In [ ]:
# boxplot - dispersão dos valores de venda por uf separado por produto
fig = px.box(dispersao_valor_venda_uf, x='uf', y='valor_venda', facet_row='produto', color='produto', boxmode='overlay')
fig.update_layout(width=2000, height=2000)
fig.show()

In [ ]:
# bar chart - média do valor de venda por produto
media_valor_venda_produto = groupby_mean(df, 'produto', 'valor_venda')

fig = px.bar(media_valor_venda_produto, x='valor_venda', y='produto', title='Média do Valor de Venda por Produto')
fig.update_layout(xaxis_range = [0.1, 6.2])
fig.show()

In [ ]:
# pie chart - distribuição da quantidade de vendas por produto
quantidade_vendas_produto = groupby_count(df, 'produto')

fig = px.pie(quantidade_vendas_produto, names='produto', values='count', title='Distribuição da Quantidade de Vendas por Produto')
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

In [ ]:
# pie chart - distribuição da quantidade de vendas por bandeira
quantidade_vendas_bandeira = groupby_count(df, 'bandeira')
top6_quantidade_vendas_bandeira = quantidade_vendas_bandeira.sort_values('count', ascending=False).head(6)

fig = px.pie(top6_quantidade_vendas_bandeira, names='bandeira', values='count', title='Distribuição da Quantidade de Vendas por Bandeira, Top 6')
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

In [ ]:
# bar chart - média do valor de venda por bandeira
media_valor_venda_bandeira = groupby_mean(df, 'bandeira', 'valor_venda')
media_valor_venda_bandeira['asc_rank_valor_venda'] = media_valor_venda_bandeira['valor_venda'].rank()
top15_media_valor_venda_bandeira = media_valor_venda_bandeira.sort_values('valor_venda', ascending=False).head(15)

fig = px.bar(top15_media_valor_venda_bandeira, x='valor_venda', y='bandeira', title='Média do Valor de Venda por Bandeira, Top 15')
fig.update_layout(xaxis_range = [0.1, 6.2])
fig.show()

In [ ]:
# table - preço médio e rank das 6 bandeiras mais consumidas
media_valor_venda_bandeira[media_valor_venda_bandeira['bandeira'].isin(top6_quantidade_vendas_bandeira['bandeira'])].sort_values('asc_rank_valor_venda')